# Part 1: Getting the data

In [1]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import lxml
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import folium
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
# Get the file

wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

fp = urllib.request.urlopen(wiki_url)
mybytes = fp.read()
mystr = mybytes.decode("utf8")
fp.close()

In [3]:
# Create the dataframe, give it proper columns

soup = BeautifulSoup(mystr)
table = str(soup.find_all("table", class_="wikitable")[0])
table
df = pd.read_html(table)[0]
df.columns = ['PostalCode', 'Borough', 'Neighborhood']

In [4]:
# Remove bad borough values
#df = df.replace('Not assigned', pd.np.nan).dropna(axis=0, how='any', subset=['Borough'])
filter = df['Borough'] != 'Not assigned'
df.where(filter, inplace=True)
df.dropna(axis=0, inplace=True)

In [5]:
# Show that we have a value where Neighbourhood is not assigned

df[df['Neighborhood'] == 'Not assigned']

,PostalCode,Borough,Neighborhood
9,M9A,Queen's Park,Not assigned


In [6]:
# Set neighbourhood = borough where neighbourhood = 'Not assigned'

mask = (df['Neighborhood'] == 'Not assighned')
df['Neighborhood'][mask]

Series([], Name: Neighborhood, dtype: object)

In [7]:
df[df['Neighborhood'] == 'Not assigned']

,PostalCode,Borough,Neighborhood
9,M9A,Queen's Park,Not assigned


In [8]:
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [9]:
df['Neighborhood'] = df.apply(
    lambda row: row['Borough'] if row['Neighborhood'] == 'Not assigned' else row['Neighborhood'], axis=1
)

In [10]:
# Fixed the problem
df[df['Borough'] == 'Queen\'s Park']

,PostalCode,Borough,Neighborhood
9,M9A,Queen's Park,Queen's Park


In [11]:
# Now to join places with the same postal code and borough

df = pd.DataFrame((df.groupby(['PostalCode','Borough'])['Neighborhood']
       .apply(lambda x: ','.join(set(x.dropna()))))).reset_index()
df 

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek"
2,M1E,Scarborough,"Morningside,Guildwood,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Richview Gardens,St. Phillips,Kingsview Villag..."
101,M9V,Etobicoke,"Jamestown,Thistletown,Albion Gardens,Humbergat..."


In [12]:
df.shape

(103, 3)

# Section 2: Appending latitude and longitude

In [13]:
# Import the CSV file

df_loc = pd.read_csv('Geospatial_Coordinates.csv')

In [14]:
# Combine the two
df = pd.concat([df, df_loc], axis=1)


In [15]:
df = df.drop(["Postal Code"], axis=1)
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Morningside,Guildwood,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Richview Gardens,St. Phillips,Kingsview Villag...",43.688905,-79.554724
101,M9V,Etobicoke,"Jamestown,Thistletown,Albion Gardens,Humbergat...",43.739416,-79.588437


# Section 3: Foursquare and mapping

Take only boroughs that contain the word Toronto

In [16]:
toronto_data = df[df.apply(lambda x: True if "Toronto" in x["Borough"] else False, axis=1)]
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"Riverdale,The Danforth West",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [17]:
# Set up url parameters

CLIENT_ID = '0V3TLIC2XB0C0WRJW5YRZXJBQEUACTOWUWUNV2N3MZZQJCC1'
CLIENT_SECRET = 'ZLHT4HW1R4IQRPBOUR3GXLLVULWXPHMB20YESYRQZ2VITVV2'
VERSION = '20180605'
radius = 500
LIMIT = 100

In [18]:
# Define the nearby venues function

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

In [20]:
print(toronto_venues.shape)
toronto_venues.head()

(1703, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
2,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
3,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


**One hot encoding so we can analyze the venue data

In [21]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[159]] + list(toronto_onehot.columns[:159]) + list(toronto_onehot.columns[160:])

toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
toronto_onehot.shape

(1703, 231)

In [23]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,Richmond,King",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.0,...,0.0,0.0,0.0,0.020000,0.0,0.0,0.01,0.0,0.01,0.0
1,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.017544,0.0,0.0,0.00,0.0,0.00,0.0
2,Business Reply Mail Processing Centre 969 Eastern,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.00,0.0
3,"CN Tower,Bathurst Quay,King and Spadina,Railwa...",0.0,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.00,0.0
4,"Cabbagetown,St. James Town",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022727,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.00,0.0


In [24]:
toronto_grouped.shape

(39, 231)

**Now it's time for some clustering!**

In [25]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=24).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 4,
       0, 0, 0, 0, 2, 3, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], dtype=int32)

In [26]:
toronto_grouped.insert(1, 'Cluster Label', kmeans.labels_)

In [27]:
clustered_labels = toronto_grouped[['Neighborhood', 'Cluster Label']]

In [28]:
toronto_merged = toronto_data.join(clustered_labels.set_index('Neighborhood'), on="Neighborhood")
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Label
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,0
41,M4K,East Toronto,"Riverdale,The Danforth West",43.679557,-79.352188,0
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4


**Let's visualize this clustering data and get out of here!**

In [30]:
latitude = 43.6532
longitude = -79.3832

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters.save("toronto_map.html")
map_clusters

If there isn't a picture of the map, a picture of the map will be hosted in the github repository

# Final Analysis

It looks like the neighborhoods in Toronto are more homegenous than one would expect. There are a couple big outliers, but the rest of the neighborhoods are more similar than they are different. This is somewhat similart to the New York data, where one of the clusters was larger than the others, but here it is far more pronounded.

Interestingly enough, all of the different clusters are further from the beach. This is to be expected, becasue costal neighborhoods have a good chance of being fundamentally different than others.